In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import os
import numpy as np

# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device('cpu')
print(f"Using device: {device}")

In [ ]:
class FacadesDataset(Dataset):
    def __init__(self, root_dir_A, root_dir_B, transform=None):
        self.root_dir_A = root_dir_A  # Directory for input images (label maps)
        self.root_dir_B = root_dir_B  # Directory for target images (facades)
        self.transform = transform
        # Get list of image paths from both directories
        self.image_paths_A = sorted([os.path.join(root_dir_A, img) for img in os.listdir(root_dir_A) if img.endswith('.jpg')])
        self.image_paths_B = sorted([os.path.join(root_dir_B, img) for img in os.listdir(root_dir_B) if img.endswith('.jpg')])

        # Ensure the number of images match
        assert len(self.image_paths_A) == len(self.image_paths_B), "Mismatch in number of images between A and B directories"
        print(f"Found {len(self.image_paths_A)} paired images in {root_dir_A} and {root_dir_B}")

    def __len__(self):
        return len(self.image_paths_A)

    def __getitem__(self, idx):
        # Load input (label map) and target (facade) images
        input_image = Image.open(self.image_paths_A[idx]).convert('RGB')
        target_image = Image.open(self.image_paths_B[idx]).convert('RGB')

        if self.transform:
            input_image = self.transform(input_image)
            target_image = self.transform(target_image)

        return input_image, target_image

In [ ]:
class FacadesInferenceDataset(Dataset):
    def __init__(self, root_dir_A, transform=None):
        self.root_dir_A = root_dir_A  # Directory for input images (label maps)
        #self.root_dir_B = root_dir_B  # Directory for target images (facades)
        self.transform = transform
        # Get list of image paths from both directories
        self.image_paths_A = sorted([os.path.join(root_dir_A, img) for img in os.listdir(root_dir_A) if img.endswith('.jpg')])
        #self.image_paths_B = sorted([os.path.join(root_dir_B, img) for img in os.listdir(root_dir_B) if img.endswith('.jpg')])

        # Ensure the number of images match
        #assert len(self.image_paths_A) == len(self.image_paths_B), "Mismatch in number of images between A and B directories"
        print(f"Found {len(self.image_paths_A)} in {root_dir_A}")

    def __len__(self):
        return len(self.image_paths_A)

    def __getitem__(self, idx):
        # Load input (label map) and target (facade) images
        input_image = Image.open(self.image_paths_A[idx]).convert('RGB')
        #target_image = Image.open(self.image_paths_B[idx]).convert('RGB')

        if self.transform:
            input_image = self.transform(input_image)
            #target_image = self.transform(target_image)

        return input_image

In [ ]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
])
print("Transformations defined: Resize to 256x256, ToTensor, Normalize to [-1, 1]")

In [ ]:
class UNetGenerator(nn.Module):
    def __init__(self):
        super(UNetGenerator, self).__init__()
        # Encoder
        self.enc1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.enc2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.enc3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.enc4 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.enc5 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.enc6 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.enc7 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.enc8 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=4, stride=2, padding=1),
            nn.ReLU(inplace=True)
        )

        # Decoder
        self.dec1 = nn.Sequential(
            nn.ConvTranspose2d(512, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5)
        )
        self.dec2 = nn.Sequential(
            nn.ConvTranspose2d(1024, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5)
        )
        self.dec3 = nn.Sequential(
            nn.ConvTranspose2d(1024, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5)
        )
        self.dec4 = nn.Sequential(
            nn.ConvTranspose2d(1024, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True)
        )
        self.dec5 = nn.Sequential(
            nn.ConvTranspose2d(1024, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True)
        )
        self.dec6 = nn.Sequential(
            nn.ConvTranspose2d(512, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True)
        )
        self.dec7 = nn.Sequential(
            nn.ConvTranspose2d(256, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )
        self.dec8 = nn.Sequential(
            nn.ConvTranspose2d(128, 3, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        e1 = self.enc1(x)
        e2 = self.enc2(e1)
        e3 = self.enc3(e2)
        e4 = self.enc4(e3)
        e5 = self.enc5(e4)
        e6 = self.enc6(e5)
        e7 = self.enc7(e6)
        e8 = self.enc8(e7)

        d1 = self.dec1(e8)
        d1 = torch.cat([d1, e7], dim=1)
        d2 = self.dec2(d1)
        d2 = torch.cat([d2, e6], dim=1)
        d3 = self.dec3(d2)
        d3 = torch.cat([d3, e5], dim=1)
        d4 = self.dec4(d3)
        d4 = torch.cat([d4, e4], dim=1)
        d5 = self.dec5(d4)
        d5 = torch.cat([d5, e3], dim=1)
        d6 = self.dec6(d5)
        d6 = torch.cat([d6, e2], dim=1)
        d7 = self.dec7(d6)
        d7 = torch.cat([d7, e1], dim=1)
        d8 = self.dec8(d7)
        return d8


In [ ]:
class PatchGANDiscriminator(nn.Module):
    def __init__(self):
        super(PatchGANDiscriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(6, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, kernel_size=4, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=1)
        )

    def forward(self, x, y):
        input = torch.cat([x, y], dim=1)  # Concatenate input and target/generated image
        return self.model(input)


In [ ]:
def weights_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif isinstance(m, nn.BatchNorm2d):
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
# Function to display images
def show_images(input_img, target_img):
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(input_img.permute(1, 2, 0) * 0.5 + 0.5)  # Denormalize
    ax[0].set_title('Input Image (Label Map)')
    ax[1].imshow(target_img.permute(1, 2, 0) * 0.5 + 0.5)
    ax[1].set_title('Target Image (Facade)')
    plt.show()



In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#data_path = "/content/drive/MyDrive/ColabNotebooks/Research/data/facade_dataset"
data_path = "/home/zia/Documents/research/data/facade"

In [ ]:
# Define the root directories for trainA and trainB (adjust based on Kaggle dataset path)
root_dir_A = data_path + '/trainA'
root_dir_B = data_path + '/trainB'
print(f"Dataset root directories set to: {root_dir_A} (inputs) and {root_dir_B} (targets)")

In [ ]:
# Load the dataset
train_dataset = FacadesDataset(root_dir_A=root_dir_A, root_dir_B = root_dir_B, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
print(f"Dataset loaded with {len(train_dataset)} samples, DataLoader created with batch size 1")

In [ ]:
# Get and display a sample
for i in range (3):

    input_img, target_img = next(iter(train_loader))
    print(f"\nDisplaying sample {i+1} from the dataset:")
    show_images(input_img[0], target_img[0])

In [ ]:
# Initialize the generator
generator = UNetGenerator().to(device)
print("U-Net Generator defined and moved to GPU")

In [ ]:
# Initialize the discriminator
discriminator = PatchGANDiscriminator().to(device)
print("PatchGAN Discriminator defined and moved to GPU")

In [ ]:
generator.apply(weights_init)
discriminator.apply(weights_init)
print("Weights initialized with normal distribution (mean=0, std=0.02) for Conv layers and (mean=1, std=0.02) for BatchNorm")

In [ ]:
# Loss functions
criterion_GAN = nn.BCEWithLogitsLoss()
criterion_L1 = nn.L1Loss()
print("Loss functions defined: BCEWithLogitsLoss for GAN, L1Loss for pixel-wise similarity")

# Optimizers
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
print("Optimizers defined: Adam with lr=0.0002, betas=(0.5, 0.999) for both Generator and Discriminator")

In [ ]:
# Test the generator with a sample input
with torch.no_grad():
    sample_input = input_img.to(device)
    sample_output = generator(sample_input)
    print(f"Generator test output shape: {sample_output.shape}")

# Visualize the initial output
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(sample_input.cpu().squeeze().permute(1, 2, 0) * 0.5 + 0.5)
ax[0].set_title('Input Image')
ax[1].imshow(sample_output.cpu().squeeze().permute(1, 2, 0) * 0.5 + 0.5)
ax[1].set_title('Initial Generated Image')
plt.show()

In [ ]:
# Training parameters
num_epochs = 200
lambda_L1 = 100
print(f"Training setup: {num_epochs} epochs, L1 lambda = {lambda_L1}")

# Start with the first epoch to demonstrate
for epoch in range(1):  # Just one epoch for now
    for i, (input_img, target_img) in enumerate(train_loader):
        input_img = input_img.to(device)
        target_img = target_img.to(device)

        # Train Discriminator
        optimizer_D.zero_grad()
        pred_real = discriminator(input_img, target_img)
        loss_D_real = criterion_GAN(pred_real, torch.ones_like(pred_real))

        generated_img = generator(input_img)
        pred_fake = discriminator(input_img, generated_img.detach())
        loss_D_fake = criterion_GAN(pred_fake, torch.zeros_like(pred_fake))

        loss_D = (loss_D_real + loss_D_fake) / 2
        loss_D.backward()
        optimizer_D.step()

        # Train Generator
        optimizer_G.zero_grad()
        pred_fake = discriminator(input_img, generated_img)
        loss_G_GAN = criterion_GAN(pred_fake, torch.ones_like(pred_fake))
        loss_G_L1 = criterion_L1(generated_img, target_img) * lambda_L1
        loss_G = loss_G_GAN + loss_G_L1
        loss_G.backward()
        optimizer_G.step()

        # Print progress every 100 steps
        if i % 100 == 0:
            print(f"Epoch 0, Step {i}/{len(train_loader)}, "
                  f"Loss_D: {loss_D.item():.4f}, Loss_G: {loss_G.item():.4f}, "
                  f"Loss_G_GAN: {loss_G_GAN.item():.4f}, Loss_G_L1: {loss_G_L1.item():.4f}")

            # Visualize
            fig, ax = plt.subplots(1, 3, figsize=(15, 5))

            ax[0].imshow(input_img.cpu().detach().squeeze().permute(1, 2, 0).numpy() * 0.5 + 0.5)
            ax[0].set_title('Input')

            ax[1].imshow(generated_img.cpu().detach().squeeze().permute(1, 2, 0).numpy() * 0.5 + 0.5)
            ax[1].set_title('Generated')

            ax[2].imshow(target_img.cpu().detach().squeeze().permute(1, 2, 0).numpy() * 0.5 + 0.5)
            ax[2].set_title('Target')

            plt.show()

In [ ]:
# Full training loop
for epoch in range(num_epochs):
    for i, (input_img, target_img) in enumerate(train_loader):
        input_img = input_img.to(device)
        target_img = target_img.to(device)

        # Train Discriminator
        optimizer_D.zero_grad()
        pred_real = discriminator(input_img, target_img)
        loss_D_real = criterion_GAN(pred_real, torch.ones_like(pred_real))

        generated_img = generator(input_img)
        pred_fake = discriminator(input_img, generated_img.detach())
        loss_D_fake = criterion_GAN(pred_fake, torch.zeros_like(pred_fake))

        loss_D = (loss_D_real + loss_D_fake) / 2
        loss_D.backward()
        optimizer_D.step()

        # Train Generator
        optimizer_G.zero_grad()
        pred_fake = discriminator(input_img, generated_img)
        loss_G_GAN = criterion_GAN(pred_fake, torch.ones_like(pred_fake))
        loss_G_L1 = criterion_L1(generated_img, target_img) * lambda_L1
        loss_G = loss_G_GAN + loss_G_L1
        loss_G.backward()
        optimizer_G.step()

        # Print progress every 100 steps
        if i % 100 == 0:
            print(f"Epoch [{epoch}/{num_epochs}], Step [{i}/{len(train_loader)}], "
                  f"Loss_D: {loss_D.item():.4f}, Loss_G: {loss_G.item():.4f}, "
                  f"Loss_G_GAN: {loss_G_GAN.item():.4f}, Loss_G_L1: {loss_G_L1.item():.4f}")

    # Visualization after each epoch (using the last batch in the epoch)
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))

    ax[0].imshow(input_img.cpu().detach().squeeze().permute(1, 2, 0).numpy() * 0.5 + 0.5)
    ax[0].set_title('Input')

    ax[1].imshow(generated_img.cpu().detach().squeeze().permute(1, 2, 0).numpy() * 0.5 + 0.5)
    ax[1].set_title('Generated')

    ax[2].imshow(target_img.cpu().detach().squeeze().permute(1, 2, 0).numpy() * 0.5 + 0.5)
    ax[2].set_title('Target')

    plt.show()

print("Training completed!")

In [ ]:
generator.eval()

# Define test dataset directories (adjust paths if necessary)
test_root_dir_A = data_path + '/testA'  # Label maps
test_root_dir_B = data_path + '/testB'  # Real facades

# Load test dataset
test_dataset = FacadesDataset(root_dir_A=test_root_dir_A, root_dir_B=test_root_dir_B, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
print(f"Test dataset loaded with {len(test_dataset)} samples")

# Initialize accumulators for metrics
total_l1_loss = 0.0
total_psnr = 0.0
num_samples = 0

# Disable gradient computation for testing
with torch.no_grad():
    for i, (input_img, target_img) in enumerate(test_loader):
        # Move images to the device (e.g., GPU)
        input_img = input_img.to(device)
        target_img = target_img.to(device)

        # Generate image using the pix2pix generator
        generated_img = generator(input_img)

        # Compute L1 loss (on normalized images in [-1, 1])
        l1_loss = criterion_L1(generated_img, target_img)
        total_l1_loss += l1_loss.item()

        # Denormalize images to [0, 1] for PSNR calculation
        generated_img_denorm = ((generated_img + 1) / 2).clamp(0, 1)
        target_img_denorm = ((target_img + 1) / 2).clamp(0, 1)

        # Compute Mean Squared Error (MSE)
        mse = torch.mean((generated_img_denorm - target_img_denorm) ** 2)

        # Compute PSNR
        if mse > 0:
            psnr = 20 * torch.log10(1.0 / torch.sqrt(mse))
            total_psnr += psnr.item()
        else:
            total_psnr += 100  # Assign a high PSNR if MSE is zero (perfect match)

        num_samples += 1

        # Visualize the first 5 examples safely
        if i < 5:
            input_img_denorm = ((input_img + 1) / 2).clamp(0, 1)

            fig, ax = plt.subplots(1, 3, figsize=(15, 5))
            ax[0].imshow(input_img_denorm.cpu().squeeze(0).permute(1, 2, 0).detach().numpy())
            ax[0].set_title('Input Label Map')

            ax[1].imshow(generated_img_denorm.cpu().squeeze(0).permute(1, 2, 0).detach().numpy())
            ax[1].set_title('Generated Facade')

            ax[2].imshow(target_img_denorm.cpu().squeeze(0).permute(1, 2, 0).detach().numpy())
            ax[2].set_title('Real Facade')

            for a in ax:
                a.axis('off')  # Hide axes for cleaner visuals
            plt.show()

# Compute average metrics
avg_l1_loss = total_l1_loss / num_samples
avg_psnr = total_psnr / num_samples

# Print the results
print(f"Test Results:")
print(f"Average L1 Loss: {avg_l1_loss:.4f}")
print(f"Average PSNR: {avg_psnr:.2f} dB")

In [ ]:
torch.save(generator, data_path + "/model.bin")

In [ ]:
# Define inference dataset directories (adjust paths if necessary)
inference_root_dir_A = data_path + '/testNapier'  # Label maps
#test_root_dir_B = data_path + '/testB'  # Real facades

# Load test dataset
inference_dataset = FacadesInferenceDataset(root_dir_A=inference_root_dir_A, transform=transform)
inference_loader = DataLoader(inference_dataset, batch_size=1, shuffle=False)
print(f"Test dataset loaded with {len(inference_dataset)} samples")

In [ ]:
generator.eval()

# Disable gradient computation for testing
num_samples = 0
with torch.no_grad():
    for i, (input_img) in enumerate(inference_loader):
        # Move images to the device (e.g., GPU)
        input_img = input_img.to(device)
        #target_img = target_img.to(device)

        # Generate image using the pix2pix generator
        generated_img = generator(input_img)

        # Compute L1 loss (on normalized images in [-1, 1])
        #l1_loss = criterion_L1(generated_img, target_img)
        #total_l1_loss += l1_loss.item()

        # Denormalize images to [0, 1] for PSNR calculation
        generated_img_denorm = ((generated_img + 1) / 2).clamp(0, 1)
        #target_img_denorm = ((target_img + 1) / 2).clamp(0, 1)

        # Compute Mean Squared Error (MSE)
        #mse = torch.mean((generated_img_denorm - target_img_denorm) ** 2)

        # Compute PSNR
        #if mse > 0:
        #    psnr = 20 * torch.log10(1.0 / torch.sqrt(mse))
        #    total_psnr += psnr.item()
        #else:
        #    total_psnr += 100  # Assign a high PSNR if MSE is zero (perfect match)

        num_samples += 1

        # Visualize the first 5 examples safely
        if i < 5:
            input_img_denorm = ((input_img + 1) / 2).clamp(0, 1)

            fig, ax = plt.subplots(1, 2, figsize=(15, 5))
            ax[0].imshow(input_img_denorm.cpu().squeeze(0).permute(1, 2, 0).detach().numpy())
            ax[0].set_title('Input Label Map')

            ax[1].imshow(generated_img_denorm.cpu().squeeze(0).permute(1, 2, 0).detach().numpy())
            ax[1].set_title('Generated Facade')

            #ax[2].imshow(target_img_denorm.cpu().squeeze(0).permute(1, 2, 0).detach().numpy())
            #ax[2].set_title('Real Facade')

            for a in ax:
                a.axis('off')  # Hide axes for cleaner visuals
            plt.show()
